<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 7.6 MB/s  eta 0:00:01
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for wget

In [2]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon
from folium import plugins

## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Mission Outcome,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),Success,No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


In [8]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for name, lat, long, in zip(launch_sites_df['Launch Site'], launch_sites_df['Lat'], launch_sites_df['Long']):
    folium.Circle([lat,long],popup = name, color = 'green', fill_color = 'blue', radius = 5000, label = launch_sites_df['Launch Site']).add_to(site_map)

for name, lat, long, in zip(launch_sites_df['Launch Site'], launch_sites_df['Lat'], launch_sites_df['Long']):
    folium.map.Marker([lat,long],  icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % name)).add_to(site_map)
                      
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


def function(marker_color):
    if spacex_df['class'] != 0:
        marker_color = 'Green'
    else:
        marker_color = 'Red'
    return marker_color

##Alternative method with list:
#marker_color= []
#for Class in spacex_df["class"]:
#    if Class ==1:
#        marker_color.append("Green")
#    else:
#        marker_color.append("Red")
#marker_color
#spacex_df["Marker Color"] = marker_color
#spacex_df.head(20)

In [12]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [13]:
# Add the Marker cluster to the site map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)
clusters = plugins.MarkerCluster().add_to(site_map)

print(len(spacex_df))

# for each row in spacex_df data frame, create a Marker object with its coordinate and customize the Marker's icon property 
#to indicate if this launch was successed or failed, e.g., icon=folium.Icon(color='white', icon_color=row['marker_color'])

for name, lat, long, color1 in zip(spacex_df['Launch Site'], spacex_df['Lat'], spacex_df['Long'], spacex_df['marker_color']):
    
    folium.Marker(
        location = [lat,long],
        icon=folium.Icon(color='white', icon_color = color1),
        popup=name
    ).add_to(clusters)
    
for name, lat, long, color1 in zip(spacex_df['Launch Site'], spacex_df['Lat'], spacex_df['Long'], spacex_df['marker_color']):
    folium.Marker([lat,long], icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0),
                    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % name)
                 ).add_to(site_map)

site_map

56


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest railway using MousePosition and calculate the distance between the railway point to the launch site.


In [16]:
# distance_railway = calculate_distance(lat1, lon1, lat2, lon2)

dist = calculate_distance(28.57299,-80.5854,28.56321,-80.57679)
dist

1.3750640035036121

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [17]:
# create and add a folium.Marker on your selected closest raiwaly point on the map
# show the distance to the launch site using the icon property 

folium.map.Marker(
    location = [28.57299,-80.5854],
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 10pt; color:blue"><b>Railway: {} Kms</b></div>'.format(round(dist,2))
        )).add_to(site_map)

site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected


In [18]:
# Create a `folium.PolyLine` object using the railway point coordinate and launch site coordinate
lat0 = [28.563197, 28.57299]
long0 = [-80.576820, -80.5854]

points=[]

for i in range(len(lat0)):
    points.append([lat0[i], long0[i]])

folium.PolyLine(points, color="blue", weight=2.5, opacity=1).add_to(site_map)

site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, coastline, highway, etc.


In [19]:
# Create a marker with distance to a closest city, coastline, highway, etc....
location1 = [34.63052, -120.62559]
location2 = [34.63709, -120.62436]
location3 = [34.63872, -120.45771]
location11 = [34.63134, -120.48245]

dist1 = calculate_distance(34.63123, -120.61031, 34.63052, -120.62559)
dist2 = calculate_distance(34.63123, -120.61031, 34.63709, -120.62436)
dist3 = calculate_distance(34.63123, -120.61031, 34.63872, -120.45771)
dist11 = calculate_distance(34.63123, -120.61031, 34.63134, -120.48245)

folium.map.Marker(location1,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 10pt; color:blue"><b>Railway: {} Kms</b></div>'.format(round(dist1,2))
        )).add_to(site_map)

folium.map.Marker(location2,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 10pt; color:coral"><b>Coastline: {} Kms</b></div>'.format(round(dist2,2))
        )).add_to(site_map)

folium.map.Marker(location3,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 10pt; color:green"><b>Highway: {} Kms</b></div>'.format(round(dist3,2))
        )).add_to(site_map)

folium.map.Marker(location11,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 10pt; color:purple"><b>City: {} Kms</b></div>'.format(round(dist11,2))
        )).add_to(site_map)

# Draw a line between the marker to the launch site

#lat2 = [34.63123, 34.63052, 34.63709, 34.63872]
#long2 = [-120.61031, -120.62559, -120.62436, -120.45771]

#points2=[]

#for i in range(len(lat2)):
    #points2.append([lat2[i], long2[i]])

#for each in points2:
    #folium.Marker(each).add_to(site_map)
    #folium.PolyLine(points2, color="blue", weight=2.5, opacity=1).add_to(site_map)

#site_map

folium.PolyLine([[34.63123, -120.61031], [34.63377, -120.62439]], color="blue", weight=2.5, opacity=1).add_to(site_map)
folium.PolyLine([[34.63123, -120.61031], [34.63709, -120.62436]], color="coral", weight=2.5, opacity=1).add_to(site_map)
folium.PolyLine([[34.63123, -120.61031], [34.63872, -120.45771]], color="green", weight=2.5, opacity=1).add_to(site_map)
folium.PolyLine([[34.63123, -120.61031], [34.63134, -120.48245]], color="purple", weight=2.5, opacity=1).add_to(site_map)

# Site CCAFS SLC
location4 = [28.56342, -80.56832]
location5 = [28.52555, -80.63461]
location9 = [28.54776, -80.79382]

dist4 = calculate_distance(28.563197, -80.576820, 28.56342, -80.56832)
dist5 = calculate_distance(28.563197, -80.576820, 28.52555, -80.63461)
dist9 = calculate_distance(28.563197, -80.576820, 28.54776, -80.79382)

folium.map.Marker(location4,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 10pt; color:coral"><b>Coast: {} Kms</b></div>'.format(round(dist4,2))
        )).add_to(site_map)

folium.map.Marker(location5,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 10pt; color:green"><b>Highway: {} Kms</b></div>'.format(round(dist5,2))
        )).add_to(site_map)

folium.map.Marker(location9,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 10pt; color:purple"><b>City: {} Kms</b></div>'.format(round(dist9,2))
        )).add_to(site_map)

folium.PolyLine([[28.563197, -80.576820], [28.56342, -80.56832]], color="coral", weight=2.5, opacity=1).add_to(site_map)
folium.PolyLine([[28.563197, -80.576820], [28.52555, -80.63461]], color="green", weight=2.5, opacity=1).add_to(site_map)
folium.PolyLine([[28.563197, -80.576820], [28.54776, -80.79382]], color="purple", weight=2.5, opacity=1).add_to(site_map)

# Site KSC LC 39A
location6 = [28.60713, -80.5945]
location7 = [28.58641, -80.61707]
location8 = [28.52481, -80.66454]
location10 = [28.56888, -80.79874]

dist6 = calculate_distance(28.57352, -80.64617, 28.60713, -80.5945)
dist7 = calculate_distance(28.57352, -80.64617, 28.58641, -80.61707)
dist8 = calculate_distance(28.57352, -80.64617, 28.52481, -80.66454)
dist10 = calculate_distance(28.57352, -80.64617, 28.56888, -80.79874)

folium.map.Marker(location6,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 10pt; color:blue"><b>Railway: {} Kms</b></div>'.format(round(dist6,2))
        )).add_to(site_map)

folium.map.Marker(location7,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 10pt; color:coral"><b>Coast: {} Kms</b></div>'.format(round(dist7,2))
        )).add_to(site_map)

folium.map.Marker(location8,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 10pt; color:green"><b>Highway: {} Kms</b></div>'.format(round(dist8,2))
        )).add_to(site_map)

folium.map.Marker(location10,
        icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 10pt; color:purple"><b>City: {} Kms</b></div>'.format(round(dist10,2))
        )).add_to(site_map)

folium.PolyLine([[28.57352, -80.64617], [28.60713, -80.5945]], color="blue", weight=2.5, opacity=1).add_to(site_map)
folium.PolyLine([[28.57352, -80.64617], [28.58641, -80.61707]], color="coral", weight=2.5, opacity=1).add_to(site_map)
folium.PolyLine([[28.57352, -80.64617], [28.52481, -80.66454]], color="green", weight=2.5, opacity=1).add_to(site_map)
folium.PolyLine([[28.57352, -80.64617], [28.56888, -80.79874]], color="purple", weight=2.5, opacity=1).add_to(site_map)

site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer\&utm_source=Exinfluencer\&utm_content=000026UJ\&utm_term=10006555\&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
